In [7]:

from src.data import get_dataloaders
from src.data import load_wili_dataset
import os

datasets = load_wili_dataset('data')
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 117500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 117500
    })
})


In [4]:
# Test on only 4 languages
languages = ['eng', 'deu', 'fra', 'nld']
for split in datasets:
    datasets[split] = datasets[split].filter(lambda example: example['label'] in languages)

Filter: 100%|██████████| 117500/117500 [00:00<00:00, 279155.72 examples/s]


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [22]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

DATA_DIR = 'data'

def train_BPE(languages):

    # create a .txt file that only contains the specific languages on which we want to train the BPE tokenizer
    output_path = os.path.join(DATA_DIR, 'BPE_train.txt')
    with open(output_path, 'w', encoding='utf-8') as output_file:
        with open(os.path.join(DATA_DIR,'x_train.txt' ), 'r', encoding='utf-8') as x_file, open(os.path.join(DATA_DIR,'y_train.txt' ), 'r', encoding='utf-8') as y_file:
            for x1, y1 in zip(x_file, y_file):
                x1 = x1.strip()
                y1 = y1.strip()
                if y1 in languages:
                    output_file.write(x1 + '\n')

    unk_token = "<UNK>"  # token for unknown words
    spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>"]
    file = [output_path]

    tokenizer = Tokenizer(BPE(unk_token = unk_token))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = BpeTrainer(special_tokens = spl_tokens)
    tokenizer.train(file, trainer)
    tokenizer.save("src/BPE_trained.json")

    return tokenizer


In [24]:
tokenizer = train_BPE(['eng'])